# Calibration resources that may be useful to the observer on duty

## Monitoring Calibration

The current calibration monitor points have not been integrated into influxdb and grafana.  You can check on the current status of calibration by pulling the relevant monitor points from etcd, shown below.

In [ ]:
import dsautils.calstatus as cs
from dsautils.dsa_store import DsaStore
ETCD = DsaStore()

In [ ]:
# Check the rsync queue.  This task rsyncs correlator output from the
# correlator machines to dsa-storage.
# All tasks should be alive (i.e. ntasks_alive == ntasks_total), and
# queue_size should not exceed 16 (the number of files for a single time,
# one generated on each correlator node).
print('rsync:', ETCD.get_dict('/mon/cal/rsync_process'))
print('')
# Check the freqeuncy integration.  This task removes the cable delays from the
# outrigger antennas before integrating in frequency by a factor of 8, down to
# 48 frequency channels per correlator node.  This acts on all 16 files for
# each time.  Again, all tasks should be alive (i.e. ntasks_alive ==
# ntasks_total), and queue_size should not exceed 16.
print('fscrunch:', ETCD.get_dict('/mon/cal/fscrunch_process'))
print('')
# Check the gathering task.  This task gathers the 16 files from the correlator
# machines for a single time. All tasks should be alive (i.e. ntasks_alive ==
# ntasks_total), and queue_size should not exceed 16. 
print('gather:', ETCD.get_dict('/mon/cal/gather_process'))
print('')
# Check the assessment queue.  This tasks checks to see if a calibrator exists
# in a given file.  It will only trigger calibration if the file contains the
# end of a 15-minute transit of the calibrator. All tasks should be alive
# (i.e. ntasks_alive == ntasks_total), and queue_size should not exceed a few.
print('assess:', ETCD.get_dict('/mon/cal/assess_process'))
print('')
# Check the calibration tasks.  This task does not run in a queue.  Instead, it
# is run by an etcd callback function that watches the /cmd/cal key.
# As a result, if multiple calls are made to /cmd/cal with the key 'calibrate'
# in short succession, multiple calibrations will be attempted simultaneous on
# the same python kernel.  This process only sends an alive signal with 60s
# cadence.
print('calibrate:', ETCD.get_dict('/mon/cal/calibrate_process'))
print('')
# Check the status of the most recent calibration. A status of -1 means that
# calibration is incomplete (perhaps in progress).  A non-negative status means
# that calibration is complete.  After calibration, plots are generated and 
# the calibration table in Grafana is updated.  The status can be decoded with
# the dsautils.calstatus.decode function.
print('most recent calibration:', ETCD.get_dict('/mon/cal/calibration'))
status = ETCD.get_dict('/mon/cal/calibration')['status']
if status > -1:
    print(cs.decode(status))

## Manual Calibration

This is an example of sending a calibration signal to the calibration pipeline if for some reason the pipeline fails and a source is not calibrated, or if you want to change, for e.g. the reference antenna or the duration of the transit that is calibrated to debug a failed calibration.  Ideally, the observer on duty will not need to do this.

In [ ]:
import glob
import os
import numpy as np
import yaml
from pkg_resources import resource_filename
import astropy.units as u
import dsautils.calstatus as cs
from dsacalib.ms_io import write_beamformer_solutions, convert_calibrator_pass_to_ms, uvh5_to_ms
from dsacalib.routines import calibrate_measurement_set, get_files_for_cal, calculate_sefd
from dsacalib.plotting import summary_plot, plot_current_beamformer_solutions
from dsacalib.calib import apply_and_correct_calibrations, apply_calibration_tables
from casacore.tables import table
import casatools as cc
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Parameters that may need to be changed
date = '2020-11-30'
calname = 'J201427+233452'
duration = 15*u.min # The length of the measurement set created, centered on the calibrator transit
refant = '102'

# Parameters that are mostly constant
calsources = resource_filename('dsacalib', 'data/calibrator_sources.csv')
antennas = np.concatenate((np.array(
    [24, 25, 26, 27, 28, 29, 30, 31, 32,
     33, 34, 35, 20, 19, 18, 17, 16, 15,
     14, 13, 100, 101, 102, 116, 103]),
    np.arange(36, 36+39)))
refcorr = '01'
filelength = 15*u.min
msdir = '/mnt/data/dsa110/calibration/'
hdf5dir = '/mnt/data/dsa110/correlator/'
msname = '{0}/{1}_{2}'.format(msdir, date, calname)
date_specifier = '{0}*'.format(date)

In [ ]:
# Get a list of the files for each calibrator
filenames = get_files_for_cal(
    calsources,
    refcorr,
    duration,
    filelength,
    date_specifier=date_specifier,
    hdf5dir=hdf5dir
)
filenames

In [ ]:
flist = ['/mnt/data/dsa110/corr{0:02d}/{1}.hdf5'.format(corr, filenames[date][calname]['files'][-1]) for corr in 
        [1, 2, 3, 21, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]]

In [ ]:
# This is the command to start calibration, if the calibration process is running
# ETCD.put_dict(
#     '/cmd/cal',
#     {
#         'cmd': 'calibrate',
#         'val':
#         {
#             'calname': calname,
#             'flist': flist
#         }
#     }
# )

In [ ]:
# You can also generate figures after the fact in the notebook
# For example - the beamformer solutions applied to the data:
plot_current_beamformer_solutions(
    filenames[date][calname]['files'],
    calname,
    date,
    corrlist=[1, 2, 3, 21, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
)

In [ ]:
for j in range(len(antennas)//10+1):
    summary_plot(
        msname,
        calname,
        2,
        ['B', 'A'],
        antennas[j*10:(j+1)*10]
    )
